In [ ]:
from IPython.display import clear_output

In [ ]:
! apt install tesseract-ocr
! apt install libtesseract-dev
!pip install pytesseract
!pip install easyocr
clear_output()

In [ ]:
import os
from google.colab import drive
from shutil import copyfile
import json

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import cv2
import pytesseract
import easyocr

import re

from PIL import Image
# from IPython.display import Image, display
from google.colab.patches import cv2_imshow

In [ ]:
print(os.getcwd())
print(os.listdir())

In [ ]:
os.chdir('/content/drive/MyDrive/Machine-Learning')
print(os.getcwd())
print(os.listdir())

In [ ]:
print("Ground truth training data")
im = Image.open(r"yolov5/runs/detect/exp12/sports-direct-914.png")
display(im)

In [ ]:
text_dict = {
    0: "1_star",
    1: "2_star",
    2: "3_star",
    3: "4_star",
    4: "5_star",
    5: "company",
    6: "coupon",
    7: "discount",
    8: "final_price",
    9: "number_rating",
    10: "original_price",
    11: "product_image",
    12: "product_title",
    13: "product_type",
}

In [ ]:
data = {key: None for key in ["product_title", "product_type", "company", "rating", "final_price", "original_price", "discount", "coupon"]}

In [ ]:
def txt_to_dict(filename):
    with open(filename, "r") as f:
        values = f.read().split()

    bbox_dict = {}
    for i in range(0, len(values), 5):
        bbox_values = values[i : i + 5]
        bbox_class = int(bbox_values[0])
        bbox_coords = [float(coord) for coord in bbox_values[1:]]
        bbox_dict[bbox_class] = bbox_coords

    return bbox_dict

In [ ]:
def visualise(image, label, box_left, box_top, box_width, box_height):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

    ax1.imshow(image)
    rect = plt.Rectangle(
        (box_left, box_top), box_width, box_height, fill=False, linewidth=2
    )
    ax1.add_patch(rect)
    props = dict(boxstyle="round", alpha=0.5)
    ax1.text(
        box_left,
        box_top - 50,
        label,
        fontsize=6,
        verticalalignment="top",
        bbox=props,
    )
    ax1.set_title('Bounding Box')

    image_array = np.array(image)
    cropped_image = image_array[box_top:box_top + box_height, box_left:box_left + box_width]

    ax2.imshow(cropped_image)
    ax2.set_title('Cropped Image')

    plt.show()

In [ ]:
import cv2
import numpy as np

def preprocess_image(image):
    if len(image.shape) == 3 and image.shape[2] == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()

    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    threshold = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 1))
    detected_lines = cv2.morphologyEx(threshold, cv2.MORPH_OPEN, horizontal_kernel, iterations=3)

    cnts, _ = cv2.findContours(detected_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for c in cnts:
        cv2.drawContours(threshold, [c], -1, (255, 255, 255), 2)

    repair_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 6))
    result = 255 - cv2.morphologyEx(255 - threshold, cv2.MORPH_CLOSE, repair_kernel, iterations=1)

    kernel = np.ones((1, 1), np.uint8)
    result = cv2.dilate(result, kernel, iterations=1)
    result = cv2.erode(result, kernel, iterations=1)
    result = cv2.threshold(cv2.bilateralFilter(result, 5, 75, 75), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    return result

In [ ]:
bboxes = txt_to_dict("yolov5/runs/detect/exp12/labels/32.txt")
print(bboxes)
print(data)
# box_center_w, box_center_h, box_width, box_height

for bbox_class, bbox_coords in bboxes.items():
    # image = Image.open("yolov5/test/sports-direct-863.png")
    image = Image.open("yolov5/test/32.png")
    image = image.convert('RGB')
    image_copy = np.uint8(image)
    # image_copy = cv2.resize(image_copy, None, fx=1.2, fy=1.2, interpolation=cv2.INTER_CUBIC)
    img_height, img_width, img_channels = image_copy.shape

    image_copy = cv2.cvtColor(image_copy, cv2.COLOR_RGB2GRAY)
    kernel = np.ones((1, 1), np.uint8)
    image_copy = cv2.dilate(image_copy, kernel, iterations=1)
    image_copy = cv2.erode(image_copy, kernel, iterations=1)
    cv2.threshold(cv2.bilateralFilter(image_copy, 5, 75, 75), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    center_w_norm = bbox_coords[0]
    center_h_norm = bbox_coords[1]
    box_width_norm = bbox_coords[2]
    box_height_norm = bbox_coords[3]
    label = text_dict[bbox_class]

    box_left = int((float(center_w_norm) - float(box_width_norm) / 2) * img_width)
    box_top = int((float(center_h_norm) - float(box_height_norm) / 2) * img_height)
    box_width = int(float(box_width_norm) * img_width)
    box_height = int(float(box_height_norm) * img_height)

    print(label, box_left, box_top, box_width, box_height)

    visualise(image, label, box_left, box_top, box_width, box_height)

    box_left = int(box_left)
    box_top = int(box_top)
    box_width = int(box_width)
    box_height = int(box_height)
    cropped_image = image_copy[
        box_top : box_top + box_height, box_left : box_left + box_width
    ]

    cropped_text = str(pytesseract.image_to_string(cropped_image.astype("uint8")))
    cropped_text = cropped_text.lstrip()
    cropped_text = cropped_text.rstrip()
    cropped_text = ' '.join(cropped_text.splitlines())
    print(cropped_text)

    if label == "original_price" or label == "final_price" or label == "discount":
        extracted_text = re.sub(r'[^0-9%()\s$]', '', cropped_text)
        extracted_text = re.sub(r'\s+', ' ', extracted_text.strip())
        data[label] == extracted_text

    label_mapping = {
        "1_star": 1,
        "2_star": 2,
        "3_star": 3,
        "4_star": 4,
        "5_star": 5
    }
    temp_star = label_mapping.get(label)

    if label == "num_rating":
        if cropped_text > 10:
            temp_rating = cropped_text / 10
        if int(temp_rating) != temp_star:
            data['rating'] = temp_rating



    data[label] = cropped_text

    with open('output_eg.json', 'w') as json_file:
        json.dump(data, json_file, indent=4)

    if label == "image":
        # cv2_imshow(cropped_image)
        # cv2.waitKey(5000)
        # cv2.destroyAllWindows()
        cv2.imwrite("cropped_image.jpg", cropped_image)